In [1]:
import json
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login


C:\Users\chuy-\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# -----------------------------
# 1. Cargar dataset
# -----------------------------
dataset = load_dataset("json", data_files="dataset.jsonl")

In [7]:
# -----------------------------
# 2. Cargar modelo base
# -----------------------------
model_name = "microsoft/Phi-3.5-mini-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="None",
)


ValueError: When passing device_map as a string, the value needs to be a device name (e.g. cpu, cuda:0) or 'auto', 'balanced', 'balanced_low_0', 'sequential' but found None.

In [4]:
# -----------------------------
# 3. Configurar LoRA
# -----------------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [5]:
# -----------------------------
# 4. Preprocesar el dataset
# -----------------------------
def format_instruction(example):
    prompt = f"Instrucción: {example['instruction']}\nRespuesta:"
    return tokenizer(prompt + example["response"], truncation=True)

tokenized = dataset.map(format_instruction)


In [6]:
# -----------------------------
# 5. Entrenamiento
# -----------------------------
training_args = TrainingArguments(
    output_dir="./lora-tutor",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    logging_steps=50,
    num_train_epochs=1,     # Pon 1 para no tardar horas
    fp16=False,             # IMPORTANTE
    bf16=False,
    save_steps=500,
    no_cuda=True,           # IMPORTANTÍSIMO
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

C:\Users\chuy-\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
The model is already on multiple devices. Skipping the move to device specified in `args`.


RuntimeError: Function MmBackward0 returned an invalid gradient at index 1 - expected device meta but got cpu

In [ ]:
# -----------------------------
# 6. Guardar adaptadores LoRA
# -----------------------------
model.save_pretrained("./lora-tutor")
print("Entrenamiento completado. Adaptadores guardados.")

In [6]:
import json
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer
import torch

# -----------------------------
# 1. Cargar dataset
# -----------------------------
dataset = load_dataset("json", data_files="tutor_dataset.jsonl")

# -----------------------------
# 2. Cargar modelo base (CPU optimizado)
# -----------------------------
model_name = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Cargar modelo en CPU con dtype float32 (sin cuantización)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,  # Usar float32 para CPU
    low_cpu_mem_usage=True,     # Optimizar uso de memoria
    device_map="cpu"            # Forzar CPU
)

# -----------------------------
# 3. Configurar LoRA (ajustado para Phi-3.5)
# -----------------------------
lora_config = LoraConfig(
    r=8,                        # Phi-3.5 mini es pequeño, podemos usar r=8
    lora_alpha=16,              
    target_modules=["qkv_proj", "o_proj"],  # Módulos correctos para Phi-3.5
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print(f"Parámetros entrenables: {model.print_trainable_parameters()}")

# -----------------------------
# 4. Preprocesar el dataset
# -----------------------------
def format_instruction(example):
    # Formato específico para Phi-3.5
    prompt = f"<|user|>\n{example['instruction']}<|end|>\n<|assistant|>\n"
    full_text = prompt + example["response"] + "<|end|>"
    
    # Tokenizar con longitud máxima
    return tokenizer(
        full_text, 
        truncation=True, 
        max_length=512,
        padding="max_length"
    )

tokenized = dataset.map(
    format_instruction, 
    remove_columns=dataset["train"].column_names
)

# -----------------------------
# 5. Entrenamiento (configuración para CPU)
# -----------------------------
training_args = TrainingArguments(
    output_dir="./lora-tutor",
    per_device_train_batch_size=2,      # Phi-3.5 es más pequeño, podemos usar batch_size=2
    gradient_accumulation_steps=8,      
    logging_steps=10,
    num_train_epochs=3,                 # 3 épocas es razonable para Phi-3.5
    fp16=False,                         
    save_steps=100,
    save_total_limit=2,                 
    learning_rate=2e-4,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    dataloader_num_workers=0,           
    optim="adamw_torch",                
    max_grad_norm=1.0,
    report_to="none",                   
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

print("Iniciando entrenamiento en CPU (esto puede ser LENTO)...")
trainer.train()

# -----------------------------
# 6. Guardar adaptadores LoRA
# -----------------------------
model.save_pretrained("./lora-tutor")
tokenizer.save_pretrained("./lora-tutor")
print("✅ Entrenamiento completado. Adaptadores guardados en ./lora-tutor")

# -----------------------------
# 7. Ejemplo de inferencia (opcional)
# -----------------------------
print("\n--- Prueba de inferencia ---")
model.eval()
test_prompt = "<|user|>\n¿Cómo crear una lista en Python?<|end|>\n<|assistant|>\n"
inputs = tokenizer(test_prompt, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.16s/it]


trainable params: 4,718,592 || all params: 3,825,798,144 || trainable%: 0.1233
Parámetros entrenables: None


Map: 100%|██████████| 79/79 [00:00<00:00, 459.39 examples/s]
The model is already on multiple devices. Skipping the move to device specified in `args`.


Iniciando entrenamiento en CPU (esto puede ser LENTO)...


C:\Users\chuy-\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,3.105000


✅ Entrenamiento completado. Adaptadores guardados en ./lora-tutor

--- Prueba de inferencia ---
¿Cómo crear una lista en Python? Crear una lista en Python es muy simple y directo. Aquí tienes una guía paso a paso y un ejemplo de cómo crear una lista:

1. **Inicializar una lista vacía**: Puedes crear una lista vacía usando corchetes `[]`.

```python
mi_lista = []
```

2. **Agregar elementos a la lista**: Puedes agregar


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# -----------------------------
# 1. Configuración
# -----------------------------
model_name = "microsoft/Phi-3.5-mini-instruct" 
lora_path = "./lora-tutor"  

# -----------------------------
# 2. Cargar modelo base y adaptadores LoRA
# -----------------------------
print("Cargando modelo base...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float32,      
    device_map="cpu",               
    low_cpu_mem_usage=True
)

print("Cargando adaptadores LoRA...")
model = PeftModel.from_pretrained(base_model, lora_path)
model.eval()  # Modo evaluación

# -----------------------------
# 3. Función de inferencia
# -----------------------------
def generar_respuesta(pregunta, max_tokens=300, temperature=0.7):
    # Formato correcto para Phi-3.5
    prompt = f"<|user|>\n{pregunta}<|end|>\n<|assistant|>\n"
    
    inputs = tokenizer(prompt, return_tensors="pt")
    # NO usar .to("cuda") porque estamos en CPU
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )
    
    respuesta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraer solo la respuesta del asistente
    if "<|assistant|>" in respuesta_completa:
        respuesta = respuesta_completa.split("<|assistant|>")[-1].strip()
    else:
        respuesta = respuesta_completa
    
    return respuesta

# -----------------------------
# 4. Ejemplos de uso
# -----------------------------
print("\n" + "="*60)
print("🤖 Tutor de Programación LoRA - Listo para responder")
print("="*60 + "\n")

# Ejemplo 1
pregunta1 = "Explícame qué es una función en Python."
print(f"Pregunta: {pregunta1}")
print(f"Respuesta: {generar_respuesta(pregunta1)}\n")

# Ejemplo 2
pregunta2 = "¿Cómo puedo leer un archivo en Python?"
print(f"Pregunta: {pregunta2}")
print(f"Respuesta: {generar_respuesta(pregunta2)}\n")

# Ejemplo 3
pregunta3 = "¿Qué es un bucle for?"
print(f"Pregunta: {pregunta3}")
print(f"Respuesta: {generar_respuesta(pregunta3)}\n")

Cargando modelo base...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:24<00:00, 12.27s/it]


Cargando adaptadores LoRA...

🤖 Tutor de Programación LoRA - Listo para responder

Pregunta: Explícame qué es una función en Python.
Respuesta: Explícame qué es una función en Python. En el contexto del lenguaje de programación Python, la palabra "función" se refiere a un bloque compuesto y reutilizable de código que realiza tareas específicas o calculos predefinidos cuando se llama con ciertos argumentos (llamados parámetros). Las funciones ayudan al desarrollo eficiente dividiendo las operaciones complejas en partes más pequeñas y manejables, permitiendo así mejor legibilidad, modularización y mantenimiento del programa.

Aquí hay algunas características clave sobre las funciones en Python:

1. Definición: Una función puede definirse usando la sintaxis `def` seguida por su nombre junto con los paréntesis para contener sus posibles argumentos entre ellos. Después viene dos puntos (:) y luego la lista de declaraciones dentro de líneas separadas u otra estructura como indentado raíz. Lo

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# -----------------------------
# 1. Configuración
# -----------------------------
model_name = "microsoft/Phi-3.5-mini-instruct" 
lora_path = "./lora-tutor"  

# -----------------------------
# 2. Cargar modelo base y adaptadores LoRA
# -----------------------------
print("Cargando modelo base...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float32,      
    device_map="cpu",               
    low_cpu_mem_usage=True
)

print("Cargando adaptadores LoRA...")
model = PeftModel.from_pretrained(base_model, lora_path)
model.eval()  # Modo evaluación

# -----------------------------
# 3. Función de inferencia
# -----------------------------
def generar_respuesta(pregunta, max_tokens=50, temperature=0.7):
    # Formato correcto para Phi-3.5
    prompt = f"<|user|>\n{pregunta}<|end|>\n<|assistant|>\n"
    
    inputs = tokenizer(prompt, return_tensors="pt")
    # NO usar .to("cuda") porque estamos en CPU
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )
    
    respuesta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraer solo la respuesta del asistente
    if "<|assistant|>" in respuesta_completa:
        respuesta = respuesta_completa.split("<|assistant|>")[-1].strip()
    else:
        respuesta = respuesta_completa
    
    return respuesta
while True:
    pregunta = input("\n👤 Tu pregunta: ").strip()
    
    if pregunta.lower() in ['salir', 'exit', 'quit']:
        print("¡Hasta luego! 👋")
        break
    
    if not pregunta:
        continue
    
    print(f"🤖 Respuesta: {generar_respuesta(pregunta)}")

C:\Users\chuy-\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargando modelo base...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.92s/it]


Cargando adaptadores LoRA...



👤 Tu pregunta:  ¿Qué es un ciclo for? y dame un ejemplo de java 


In [ ]:
import json
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer
import torch
import gc

print("Entrenamiento LoRA con Qwen 2.5-0.5B")
print("="*70)

# -----------------------------
# 1. Cargar dataset
# -----------------------------
print("\nCargando dataset...")
dataset = load_dataset("json", data_files="tutor_dataset2.jsonl")
print(f" Dataset cargado: {len(dataset['train'])} ejemplos")

# -----------------------------
# 2. Cargar modelo base LIGERO
# -----------------------------
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
print(f"\n🤖 Cargando modelo: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Configurar padding token si no existe
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("⚙️  Cargando modelo base...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,      
    device_map="cpu",
    low_cpu_mem_usage=True,
    trust_remote_code=True
)

# Liberar memoria
gc.collect()
print(f" Modelo cargado en CPU con dtype float16")

# -----------------------------
# 3. Configurar LoRA (optimizado para Qwen)
# -----------------------------
print("\n🔧 Configurando LoRA...")

lora_config = LoraConfig(
    r=16,                           
    lora_alpha=32,                  
    target_modules=[
        "q_proj", 
        "k_proj", 
        "v_proj", 
        "o_proj"
    ],                              
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print(" LoRA configurado")
model.print_trainable_parameters()

# -----------------------------
# 4. Preprocesar el dataset
# -----------------------------
print("\n📝 Preprocesando dataset...")

def format_instruction(example):
    """
    Formato de chat para Qwen 2.5:
    <|im_start|>user
    {pregunta}<|im_end|>
    <|im_start|>assistant
    {respuesta}<|im_end|>
    """
    prompt = f"<|im_start|>user\n{example['instruction']}<|im_end|>\n<|im_start|>assistant\n"
    full_text = prompt + example["response"] + "<|im_end|>"
    
    # Tokenizar con longitud controlada
    return tokenizer(
        full_text, 
        truncation=True, 
        max_length=512,             # Límite razonable para CPU
        padding="max_length"
    )

tokenized = dataset.map(
    format_instruction, 
    remove_columns=dataset["train"].column_names,
    desc="Tokenizando ejemplos"
)

print(f"✅ Dataset tokenizado: {len(tokenized['train'])} ejemplos")

# Liberar memoria
gc.collect()

# -----------------------------
# 5. Configuración de entrenamiento
# -----------------------------
print("\n⚙️  Configurando entrenamiento...")

training_args = TrainingArguments(
    output_dir="./lora-qwen-tutor",
    per_device_train_batch_size=2,      
    gradient_accumulation_steps=8,      
    logging_steps=10,
    num_train_epochs=3,                 
    fp16=False,                         
    bf16=False,                         
    save_steps=100,
    save_total_limit=2,                 
    learning_rate=3e-4,                 
    warmup_steps=50,
    lr_scheduler_type="cosine",
    dataloader_num_workers=0,           
    optim="adamw_torch",                
    max_grad_norm=1.0,
    report_to="none",                   
    logging_dir="./logs",
    save_strategy="steps",
    eval_strategy="no",           
)

# -----------------------------
# 6. Crear Trainer
# -----------------------------
print("\n🏋️  Creando Trainer...")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# -----------------------------
# 7. ENTRENAR
# -----------------------------
print("\n" + "="*70)
print(" INICIANDO ENTRENAMIENTO")
print("="*70)
print("  ADVERTENCIA: En CPU esto puede tardar 30 min - 2 horas")
print("    dependiendo de tu procesador y tamaño del dataset")
print("="*70 + "\n")

try:
    trainer.train()
    print("\n Entrenamiento completado!")
except KeyboardInterrupt:
    print("\n  Entrenamiento interrumpido por el usuario")
except Exception as e:
    print(f"\n Error durante el entrenamiento: {e}")

# -----------------------------
# 8. Guardar modelo
# -----------------------------
print("\n💾 Guardando adaptadores LoRA...")
model.save_pretrained("./lora-qwen-tutor")
tokenizer.save_pretrained("./lora-qwen-tutor")
print("✅ Adaptadores guardados en: ./lora-qwen-tutor")

# Liberar memoria
del model
del trainer
gc.collect()

print("\n" + "="*70)
print(" PROCESO COMPLETADO")

Entrenamiento LoRA con Qwen 2.5-0.5B

Cargando dataset...
 Dataset cargado: 500 ejemplos

🤖 Cargando modelo: Qwen/Qwen2.5-0.5B-Instruct
⚙️  Cargando modelo base...
 Modelo cargado en CPU con dtype float16

🔧 Configurando LoRA...
 LoRA configurado
trainable params: 2,162,688 || all params: 496,195,456 || trainable%: 0.4359

📝 Preprocesando dataset...
✅ Dataset tokenizado: 500 ejemplos

⚙️  Configurando entrenamiento...

🏋️  Creando Trainer...


The model is already on multiple devices. Skipping the move to device specified in `args`.



 INICIANDO ENTRENAMIENTO
  ADVERTENCIA: En CPU esto puede tardar 30 min - 2 horas
    dependiendo de tu procesador y tamaño del dataset



C:\Users\chuy-\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
